# Multi-Agent Workflow for Control-M Job Conversion to Ansible Playbook

This notebook demonstrates an automated multi-agent workflow that converts a Control-M job definition into an Ansible playbook. The workflow also performs linting, review, and validation of the generated playbook using multiple agents, ensuring it follows best practices. The process includes:

1. **Job Conversion**: A Control-M job is converted into an Ansible playbook format.
2. **Playbook Linting**: The playbook is checked for errors and inconsistencies using `ansible-lint`.
3. **Playbook Review**: The playbook is reviewed and validated based on the original Control-M job.
4. **Feedback Loop**: If the playbook fails the review, the workflow attempts to adjust and reconvert the job until it passes validation or reaches a defined limit of iterations.

This notebook leverages `langgraph`, `RoleAgent`, and a multi-agent state graph to orchestrate the entire process, providing an efficient method to convert and validate automation jobs in a scalable and repeatable manner.

## 1. Install Required Dependencies

Installs all the necessary libraries to handle HTTP requests, schema validation, error handling, graph management, and terminal color output.

In [122]:
%pip install requests jsonschema tenacity langgraph langgraph.checkpoint.sqlite termcolor ansible-lint

Note: you may need to restart the kernel to use updated packages.


## 2. Initialize the Model Service

This initializes a model service, using the latest version of "llama3.1" to manage AI-driven tasks.

In [123]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:latest")

## 3. Define Graph State for Multi-Agent Workflow

Defines the structure of the state that tracks job, playbook, linting results, and feedback for the workflow using type annotations.

In [124]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any

class MultiAgentGraphState(TypedDict):
    job: str
    playbook: Annotated[list, add_messages]
    lint: Annotated[list, add_messages]
    feedback: Annotated[list, add_messages]

## 4. Example Control-M Job

This is an example of a Control-M job configuration, which includes commands and execution settings.


In [125]:
controlm_job = """
"JobName": {
    "Type" : "Job:Command",
    "Command" : "echo hello",
    "PreCommand": "echo before running main command",
    "PostCommand": "echo after running main command",
    "Host" : "myhost.mycomp.com",
    "RunAs" : "user1"  
}
"""

## 5. Convert Control-M Job to Playbook

Converts the Control-M job into an Ansible playbook using the `convert_agent`. The playbook is formatted as YAML and saved locally.

In [126]:
from agent.role_agent import RoleAgent
from prompt.convert_prompt import DEFAULT_SYS_DEV_PROMPT
import yaml
import json
import os

def convert_agent(user_req: str, feedback: str = ""):
    convert_agent = RoleAgent(
        role="CONVERT_AGENT",
        ollama_service=ollama_service,
        sys_prompt=DEFAULT_SYS_DEV_PROMPT.format(
            feedback=feedback,
            datetime=get_current_utc_datetime(),
        ),
    )

    response = convert_agent.work(user_request=user_req)

    playbook_json = json.loads(response["CONVERT_AGENT_response"].content)

    # Ensure the playbook is in an array format
    playbook_json_in_array = [playbook_json]  # Wrap the playbook in an array   

    playbook_yaml = yaml.dump(
        playbook_json_in_array,
        default_flow_style=False,  # Preserves block-style formatting
        indent=2,  # Sets the indentation level to 2 spaces
        width=80,  # Sets the max line width (optional)
        allow_unicode=True,  # Handles special characters properly
    )

    # Save the YAML to data/playbook.yaml
    playbook_path = "data/playbook.yaml"
    os.makedirs(
        os.path.dirname(playbook_path), exist_ok=True
    )  # Create the directory if it doesn't exist
    with open(playbook_path, "w") as file:
        file.write(playbook_yaml)
    return {"playbook": playbook_yaml}

## 6. Lint the Ansible Playbook

Runs `ansible-lint` on the generated playbook to ensure it meets Ansible best practices. Outputs the results.

In [127]:
import subprocess

def ansible_lint(state: dict):
    # Run the ansible-lint command and capture its output
    result = subprocess.run(
        ["ansible-lint", "data/playbook.yaml"], capture_output=True, text=True
    )
    
    # Print the standard output and standard error
    print("Standard Output:")
    print(result.stdout)

    print("\nStandard Error:")
    print(result.stderr)
    
    return {"lint":result.stdout}

## 7. Review Playbook

Uses the `reviewer_agent` to review the playbook, taking into account the Control-M job, playbook, and lint feedback. Returns structured feedback.

In [128]:
from prompt.review_prompt import DEFAULT_SYS_REVIEW_PROMPT
from utils.general.helpers import get_current_utc_datetime

def reviewer_agent(controlm_job: str, playbook_yaml: str, lint: str = ""):
    reviewer_agent = RoleAgent(
        role="REVIEWER_AGENT",
        ollama_service=ollama_service,
        sys_prompt=DEFAULT_SYS_REVIEW_PROMPT.format(
            controlm_job=controlm_job,
            lint=lint,
            datetime=get_current_utc_datetime(),
        ),
    )

    reviewer_req = f"""Validate this playbook: {playbook_yaml}"""

    response = reviewer_agent.work(user_request=reviewer_req)

    revision_json = json.loads(response["REVIEWER_AGENT_response"].content)

    return {"feedback": json.dumps(revision_json, indent=4)}

## 8. Conditional Workflow Check

Determines if the playbook is valid based on feedback. If valid, it ends the workflow; otherwise, it reverts to converting the job.

In [129]:
from langgraph.graph import END

def should_continue(state: MultiAgentGraphState):
    
    feedback_dict = json.loads(state["feedback"][-1].content)
    status = feedback_dict["status"]

    if status == "valid":
        return END
    else:
        return "convert_job"

## 9. Create the Multi-Agent Graph Workflow

Creates the flow of the multi-agent graph, linking conversion, linting, and review nodes. Defines conditional transitions based on the review status.

In [130]:
from langgraph.graph import StateGraph, START, END


def create_graph() -> StateGraph:
    graph = StateGraph(MultiAgentGraphState)

    graph.add_node(
        "convert_job",
        lambda state: convert_agent(
            user_req=f"""Convert this Control-M Job to an Ansible playbook. {state['job']}""",
            feedback=state["feedback"][-1].content if state["feedback"] else None,
        ),
    )

    graph.add_node(
        "ansible_lint", ansible_lint
    )

    graph.add_node(
        "review_playbook",
        lambda state: reviewer_agent(
            controlm_job=state["job"],
            playbook_yaml=state["playbook"][-1].content,
            lint=state["lint"][-1].content,
        ),
    )

    # Define the flow of the graph
    graph.add_edge(START, "convert_job")
    graph.add_edge("convert_job", "ansible_lint")
    graph.add_edge("ansible_lint", "review_playbook")
    graph.add_conditional_edges(
        "review_playbook", should_continue, ["convert_job", END]
    )

    return graph

## 10. Initialize Memory and Compile Workflow

Initializes an in-memory database to store workflow checkpoints. Then, compiles the multi-agent graph into a runnable workflow.

In [131]:
from memory.sqlite_saver import initialize_memory

memory = initialize_memory(":memory:")

config = {"configurable": {"thread_id": "1"}}
dict_inputs = {"job": controlm_job}

# Create the graph and compile the workflow
graph = create_graph()
workflow = graph.compile(checkpointer=memory)
print("Graph and workflow created.")

Graph and workflow created.


## 11. Execute Workflow and Track Progress

Executes the compiled workflow with a set number of iterations. Prints output with colored status updates, showing the current state of the workflow at each step.

In [132]:
from termcolor import colored

# Define workflow parameters
iterations = 10
verbose = True

limit = {"recursion_limit": iterations}

print(colored(controlm_job,"blue"))

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
        if "convert_job" in event:
            print(colored(event["convert_job"]["playbook"], "yellow"))
        elif "review_playbook" in event:
            dict = json.loads(event["review_playbook"]["feedback"])
            if(dict["status"] == "valid"):
                print(colored(event["review_playbook"]["feedback"], "green"))
            else:
                print(colored(event["review_playbook"]["feedback"], "red"))
    else:
        print("\n")


"JobName": {
    "Type" : "Job:Command",
    "Command" : "echo hello",
    "PreCommand": "echo before running main command",
    "PostCommand": "echo after running main command",
    "Host" : "myhost.mycomp.com",
    "RunAs" : "user1"  
}

- become: true
  become_user: user1
  gather_facts: false
  hosts: myhost.mycomp.com
  tasks:
  - name: pre command
    shell: echo 'before running main command'
  - name: main command
    shell: echo hello
  - name: post command
    shell: echo 'after running main command'

Standard Output:
]8;id=851535;https://ansible.readthedocs.io/projects/lint/rules/name/\name[play]]8;;\: All plays should be named.
data/playbook.yaml:1

]8;id=539323;https://ansible.readthedocs.io/projects/lint/rules/command-instead-of-shell/\command-instead-of-shell]8;;\: Use shell only when shell functionality is required.
data/playbook.yaml:6 Task/Handler: pre command

]8;id=591385;https://ansible.readthedocs.io/projects/lint/rules/fqcn/\fqcn[action-core]]8;;\: Us